In [1]:
# Import packages

import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

Matplotlib is building the font cache; this may take a moment.


In [38]:
def clean_transform(df):
    # Filter and immediately create a copy to avoid SettingWithCopyWarning
    df = df[df['type'].str.contains('HKQuantityTypeIdentifier|HKWorkoutTypeIdentifier', na=False)].copy()
    
    # Convert dates
    df['startDate'] = pd.to_datetime(df['startDate'])
    df['endDate'] = pd.to_datetime(df['endDate'])

    # Extract date parts
    df['date'] = df['startDate'].dt.date
    df['month'] = df['startDate'].dt.to_period('M')
    df['year'] = df['startDate'].dt.year
    df['weekday'] = df['startDate'].dt.day_name()

    # Keep only necessary columns
    keep_cols = ['type', 'sourceName', 'unit', 'value', 'startDate', 'endDate', 'date', 'month', 'year']
    df = df[keep_cols]

    # Extract readable workout/metric type name
    df['workout_type'] = df['type'].str.extract(r'Identifier(.+)$')

    # Ensure numeric values
    df['value'] = pd.to_numeric(df['value'], errors='coerce')

    return df

In [40]:
# Update with your actual file path
xml_path = '/Users/steph/anaconda_projects/apple-health-data/raw-data/apple_health_export_2025-04-18/export.xml'
df = load_and_prepare_health_data(xml_path)

/var/folders/66/9mnm678944b64gn93ncc5cfw0000gp/T/ipykernel_48061/1789964194.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['startDate'].dt.to_period('M')


In [41]:
# Preview df
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9268187 entries, 0 to 9338868
Data columns (total 10 columns):
 #   Column        Dtype                    
---  ------        -----                    
 0   type          object                   
 1   sourceName    object                   
 2   unit          object                   
 3   value         float64                  
 4   startDate     datetime64[ns, UTC-07:00]
 5   endDate       datetime64[ns, UTC-07:00]
 6   date          object                   
 7   month         period[M]                
 8   year          int32                    
 9   workout_type  object                   
dtypes: datetime64[ns, UTC-07:00](2), float64(1), int32(1), object(5), period[M](1)
memory usage: 742.5+ MB


In [ ]:
# Format current date to use in file path 
current_date = datetime.today().strftime('%Y-%m-%d')

filename = f'/Users/steph/anaconda_projects/apple-health-data/cleaned-data/apple_health_data_full_export_{current_date}.csv'

# Save as CSV
df.to_csv(filename, index=False)